<a href="https://colab.research.google.com/github/UN-GCPDS/python-gcpds.EEG_Tensorflow_models/blob/main/Experimental/CNN_DW_ITL/Cuaderno_Gauss_y_CSP%2BCWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cargar zip de los sujetos y funciones

In [ ]:

from pydrive.drive import GoogleDrive
import io
import zipfile
drive = GoogleDrive()
import os
import numpy as np
#from CNN_Diego import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt
#--------------------------------------------------------------------------------------
!pip install tf-keras-vis tensorflow
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import pickle

     |████████████████████████████████| 53 kB 2.0 MB/s 


## Cargar CSP y CWT


In [ ]:
#CSP + CWT GIGA 1 ventana, 4 bandas de frecuencia
#url https://drive.google.com/file/d/1zvvtLD4x7TA340uKWv4YlNId8EbtQ14c/view?usp=sharing
FILEID = "1zvvtLD4x7TA340uKWv4YlNId8EbtQ14c"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O cspycwt.zip && rm -rf /tmp/cookies.txt
!unzip cspycwt.zip

## Cargar Gauss

In [ ]:
#Gauss 1 ventana, 4 bandas de frecuencia
#url https://drive.google.com/file/d/1vqgbL06dKw-g7vNH0rQ2rvBL4n0BzkSw/view?usp=sharing
FILEID = "1vqgbL06dKw-g7vNH0rQ2rvBL4n0BzkSw"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O gauss.zip && rm -rf /tmp/cookies.txt
!unzip gauss.zip

# Red CNN D&W

In [ ]:
import tempfile
import numpy as np
import pandas as pd
from scipy import stats
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import ShuffleSplit
from sklearn.utils.multiclass import unique_labels
from sklearn.cluster import KMeans, SpectralClustering
 

def conv_to_list(arr):
    x_pre = []
    for i in range(arr.shape[-1]):
        x_pre.append(arr[:, :, :, :, i])
    return x_pre
  
def labels_convert(labels_pre, labels_true):
    labels_conv = np.zeros(labels_pre.shape)
    u = np.unique(labels_pre)
    ve = []
    for i in u:
        val = stats.mode(labels_true[labels_pre == i], 0)[0]
        ve.append(val[0])
        labels_conv[labels_pre == i] = val
    return labels_conv.astype('int')
 
def plot_model_(history_):
    pd.DataFrame(history_.history).plot(figsize=(5, 5))
    plt.grid(True)
    plt.show()
 
class CNNrITL2(BaseEstimator):
  def __init__(self, d=2, sigma=None, k=2, verbose=10, n_fil=1,
                epochs=200, batch_size=128, lr=1e-3, sl='ritl',
                lk=0.5, l1=1e-3, wi=True, sbj=1, plot_model=True):
    self.verbose = verbose
    self.sigma = sigma
    self.d = d
    self.n_fil = n_fil
    self.k = k
    self.epochs = epochs
    self.batch_size = batch_size
    self.lr = lr
    self.sl = sl
    self.lk = lk
    self.l1 = l1
    self.sbj = sbj
    self.wi = wi
    self.plot_model = plot_model
      

  def transform(self, X, *_):
    return self.model.predict(X)

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(x)

  def fit(self, X, y=None):
    self.model = self.main(X, y)
    return self

  def main(self, x, y):

    # --------------------------------------------------
    # deep and wide mode for BCI topoplots
    if self.wi:
      print(x.shape)
      #batch0_ = [None] * x.shape[-1]
      input_ = [None] * x.shape[-1]
      conv0_ = [None] * x.shape[-1]
      pool0_ = [None] * x.shape[-1]
      batch1_ = [None] * x.shape[-1]
      batch2_ = [None] * x.shape[-1]
      conv1_ = [None] * x.shape[-1]
      pool1_ = [None] * x.shape[-1]
      batch3_ = [None] * x.shape[-1]
      batch4_ = [None] * x.shape[-1]
      conv2_ = [None] * x.shape[-1]
      pool2_ = [None] * x.shape[-1]
      batch5_ = [None] * x.shape[-1]
      batch6_ = [None] * x.shape[-1]
      for i in range(x.shape[-1]):
        input_[i] = keras.layers.Input(shape=x.shape[1:-1])
        #batch0_[i] = keras.layers.BatchNormalization()(input_[i])
        # ------------------------------------------------------------------------------------------------------
        conv0_[i] = keras.layers.Conv2D(filters=self.n_fil, kernel_size=5, strides=1, activation='relu',
                                        padding='SAME', input_shape=x.shape[1:-1],
                                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),
                                        kernel_initializer="GlorotNormal",name='C1_bf_'+str(i))(input_[i])
        batch1_[i] = keras.layers.BatchNormalization()(conv0_[i])
        pool0_[i] = keras.layers.MaxPooling2D(pool_size=2)(batch1_[i])
        batch2_[i] = keras.layers.BatchNormalization()(pool0_[i])
        # ------------------------------------------------------------------------------------------------------
        conv1_[i] = keras.layers.Conv2D(filters=int(self.n_fil*0.5), kernel_size=5, strides=1, activation='relu',
                                        padding='SAME', input_shape=x.shape[1:-1],
                                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),
                                        kernel_initializer="GlorotNormal",name='C2_bf_'+str(i))(batch2_[i])
      
        batch3_[i] = keras.layers.BatchNormalization()(conv1_[i])
        pool1_[i] = keras.layers.MaxPooling2D(pool_size=2)(batch3_[i])
        batch4_[i] = keras.layers.BatchNormalization()(pool1_[i])
        # ------------------------------------------------------------------------------------------------------
        conv2_[i] = keras.layers.Conv2D(filters=int(self.n_fil*0.25), kernel_size=5, strides=1, activation='relu',
                                        padding='SAME', input_shape=x.shape[1:-1],
                                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),
                                        kernel_initializer="GlorotNormal",name='C3_bf_'+str(i))(batch4_[i])
      
        batch5_[i] = keras.layers.BatchNormalization()(conv2_[i])
        pool2_[i] = keras.layers.MaxPooling2D(pool_size=2)(batch5_[i])
        batch6_[i] = keras.layers.BatchNormalization()(pool2_[i])
        # ------------------------------------------------------------------------------------------------------

      concat = keras.layers.concatenate(batch6_)
      flat = keras.layers.Flatten()(concat)
      dim = np.round(self.d * batch6_[i].shape[1] * batch6_[i].shape[2])
    else:  # only deep mode
      input_ = keras.layers.Input(shape=x.shape[1:])
      # ----------------------------------------------------------------------------------------------------------
      conv_ = keras.layers.Conv2D(filters=self.n_fil, kernel_size=5, strides=1, activation='relu',
                                  padding='SAME', input_shape=x.shape[1:],
                                  kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),name='C1_')(input_)
      batch1_ = tf.keras.layers.BatchNormalization()(conv_)
      pool_ = keras.layers.MaxPooling2D(pool_size=2)(batch1_)
      batch2_ = tf.keras.layers.BatchNormalization()(pool_)
      # ----------------------------------------------------------------------------------------------------------
      conv1_ = keras.layers.Conv2D(filters=self.n_fil, kernel_size=5, strides=1, activation='relu',
                                  padding='SAME', input_shape=x.shape[1:],
                                  kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),name='C2_')(batch2_)
      batch3_ = tf.keras.layers.BatchNormalization()(conv1_)
      pool1_ = keras.layers.MaxPooling2D(pool_size=2)(batch3_)
      batch4_ = tf.keras.layers.BatchNormalization()(pool1_)
      
      # ----------------------------------------------------------------------------------------------------------
      conv2_ = keras.layers.Conv2D(filters=self.n_fil, kernel_size=5, strides=1, activation='relu',
                                  padding='SAME', input_shape=x.shape[1:],
                                  kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1),name='C3_')(batch4_)
      batch5_ = tf.keras.layers.BatchNormalization()(conv2_)
      pool2_ = keras.layers.MaxPooling2D(pool_size=2)(batch5_)
      batch6_ = tf.keras.layers.BatchNormalization()(pool2_)
      
      # ----------------------------------------------------------------------------------------------------------
      flat = keras.layers.Flatten()(batch6_)
      dim = np.round(self.d * flat.shape[1])
    # multilayer dense  perceptron with rff
    # -----------------------------------------------------------------------------------------
    flat_do = tf.keras.layers.Dropout(rate=0.25)(flat)
    d1 = tf.keras.layers.Dense(dim.astype('int'), activation='linear', kernel_initializer="GlorotNormal",
                                kernel_regularizer=regularizers.l1_l2(l1=self.l1, l2=self.l1))(flat_do)
    h1 = tf.keras.layers.experimental.RandomFourierFeatures(output_dim=dim.astype('int'),
                                                            scale=self.sigma, kernel_initializer='gaussian',
                                                            trainable=True, name='rbf_fourier')(d1)
    h1_bn = tf.keras.layers.BatchNormalization(name='brff')(h1)
    output_prob = tf.keras.layers.Dense(self.k, activation='softmax', name='out', kernel_initializer="GlorotNormal",
                                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=self.l1, l2=self.l1))(
        h1_bn)
    # define loss and compile
    opt = tf.keras.optimizers.Adam(learning_rate=self.lr)
    if self.sl == 'ritl':
      model = tf.keras.Model(inputs=input_, outputs=[output_prob, h1_bn])
      model_loss = [tf.keras.losses.CategoricalCrossentropy(), self.custom_ritl()]
      model.compile(loss=model_loss, loss_weights=[self.lk, 1 - self.lk], optimizer=opt,
                    metrics=['accuracy'])  # f1, precision, re
    elif self.sl == 'ce':
      model = tf.keras.Model(inputs=input_, outputs=output_prob)
      model_loss = [tf.keras.losses.CategoricalCrossentropy()]
      model.compile(loss=model_loss, optimizer=opt, metrics=['accuracy'])  # f1, precision, re
    else:
      model = tf.keras.Model(inputs=input_, outputs=output_prob)
      model_loss = [tf.keras.losses.MeanSquaredError()]
      model.compile(loss=model_loss, optimizer=opt, metrics=['accuracy'])

    rs = ShuffleSplit(n_splits=1, test_size=.1)

    # validation data
    for train_index, valid_index in rs.split(x):
      if self.wi:

          x_pre = conv_to_list(x[train_index])
          x_pre_v = conv_to_list(x[valid_index])

      else:
          x_pre = x[train_index]
          x_pre_v = x[valid_index]

    y_pre = keras.utils.to_categorical(y, self.k)  #

    if self.sl == 'ritl':
      history = model.fit(x_pre, [y_pre[train_index], y_pre[train_index]],
                          epochs=self.epochs,
                          batch_size=self.batch_size,
                          validation_data=(x_pre_v, [y_pre[valid_index], y_pre[valid_index]]),
                          verbose=self.verbose)
    else:
      history = model.fit(X_pre, y_pre[train_index],
                          epochs=self.epochs,
                          batch_size=self.batch_size,
                          validation_data=(X_pre_v, y_pre[valid_index]),
                          verbose=self.verbose)
    if self.plot_model:
      plot_model_(history)

    return model

  def predict(self, x_test):

    if self.wi:
      x_test_pre = conv_to_list(x_test)
    else:
      x_test_pre = x_test

    if self.sl == 'ritl':
      y_prob = np.stack(
          [self.model(x_test_pre, training=True)[0]
            for sample in range(100)])
    else:
      y_prob = np.stack(
          [self.model(x_test_pre, training=True)
            for sample in range(100)])

    y_prob = y_prob.mean(axis=0)
    label_e = np.argmax(y_prob, axis=1)

    return label_e

  def custom_ritl(self):
    def custom_kitl(y_true, y_pred):
      # -------------------------kernel----------------------------
      k = tf.matmul(y_pred, y_pred, transpose_b=True)
      # ----------------------center-------------------------------
      N = tf.cast(tf.shape(k)[0], dtype=tf.float32)
      # matrix for centered kernel
      h = tf.eye(N) - (1.0 / N) * tf.ones([N, 1]) * tf.ones([1, N])
      k = tf.matmul(tf.matmul(k, h), tf.matmul(k, h))
      k = tf.math.divide_no_nan(k, tf.linalg.trace(k))
      # ------------------------F_initial--------------------------
      f = -tf.math.log(tf.linalg.trace(tf.matmul(k, k) + 1E-10))
      return -f

    return custom_kitl

  def get_params(self, deep=True):

    return {'d': self.d, 'k': self.k, 'epochs': self.epochs, 'batch_size': self.batch_size,
            'lr': self.lr, 'sl': self.sl, 'lk': self.lk, 'l1': self.l1,
            'wi': self.wi, 'sbj': self.sbj, 'verbose': self.verbose, 'n_fil': self.n_fil,
            'plot_model': self.plot_model}

  def set_params(self, **parameters):
    for parameter, value in parameters.items():
      setattr(self, parameter, value)
    return self

## Cargar sujetos para csp y cwt


In [ ]:
try:
  os.makedirs('/content/csp y cwt/Modelos/')
except:
  print('ya esxiste la carpeta')

In [ ]:
aux = np.arange(52)+1
subjects = np.delete(aux,[28,33])
for sbj in subjects:
  print('Sujeto: ',sbj)  
  Xcsp = np.load('/content/csp y cwt/csp/Fold1/train/sbj'+str(sbj)+'.npy')
  Xcwt = np.load('/content/csp y cwt/cwt/Fold1/train/sbj'+str(sbj)+'.npy')
  X_train = np.expand_dims(np.concatenate([np.moveaxis(Xcsp, -1, 3).reshape(Xcsp.shape[0], Xcsp.shape[1], Xcsp.shape[2], -1 ), np.moveaxis(Xcwt, -1, 3).reshape(Xcwt.shape[0], Xcwt.shape[1], Xcwt.shape[2], -1 )], axis=-1), axis=3)
  y_train = np.load('/content/csp y cwt/Targets/Fold1/trainsbj'+str(sbj)+'.npy')

  Xcsp = np.load('/content/csp y cwt/csp/Fold1/test/sbj'+str(sbj)+'.npy')
  Xcwt = np.load('/content/csp y cwt/cwt/Fold1/test/sbj'+str(sbj)+'.npy')
  X_test = np.expand_dims(np.concatenate([np.moveaxis(Xcsp, -1, 3).reshape(Xcsp.shape[0], Xcsp.shape[1], Xcsp.shape[2], -1 ), np.moveaxis(Xcwt, -1, 3).reshape(Xcwt.shape[0], Xcwt.shape[1], Xcwt.shape[2], -1 )], axis=-1), axis=3)
  y_test = np.load('/content/csp y cwt/Targets/Fold1/testsbj'+str(sbj)+'.npy')

  X = np.concatenate((X_train,X_test),axis=0)
  y = np.concatenate((y_train, y_test))
  #---------------------------------------------------------------------------------
  # Correr solo el modelo
  #P= CNNrITL(d=2, k=len(np.unique(y)), epochs=150, batch_size=64, 
  #           lr=1e-3, sl='ritl', lk=0.5, l1=1e-3, plot_model=True)
  #P.fit(X,y)
  #--------------------------------------------------------------------------------
  # Correr con grid search
  steps = [('proy', CNNrITL2(epochs=150, batch_size=64, k=len(np.unique(y)), lr = 1e-3, sl='ritl')),
           ]    
  method = Pipeline(steps ,memory='datospipeline')
  parameters ={ 'proy__l1':[1e-3, 1e-2, 1e-1],
                'proy__lk':[0.5,0.75,1],
                'proy__d':[2,5,10],
                }
  cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=123)
  grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
                      scoring='accuracy',refit='accuracy',n_jobs=-1,error_score='raise' )
  grid_search.fit(X, y)
  print(grid_search.best_params_)
  # guardar modelo 
  grid_search.best_estimator_[0].model.save('/content/csp y cwt/Modelos/sbj'+str(sbj)+'.h5')
  print('para el sujeto {} el accuracy  es {}'.format(sbj,grid_search.best_score_))

Sujeto:  1
Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 48.9min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 69.7min


KeyboardInterrupt: ignored

## Cargar sujetos para gauss

In [ ]:
try:
  os.makedirs('/content/gauss/Modelos/')
except:
  print('ya esxiste la carpeta')

In [ ]:
aux = np.arange(52)+1
subjects = np.delete(aux,[28,33])
for sbj in subjects:
  print(sbj)
  X = np.load('/content/gauss/Gaussian_Kernel/sbj'+str(sbj)+'.npy')
  X = np.moveaxis(X, -1, 4).reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], -1)
  y = np.load('/content/gauss/Targets/sbj'+str(sbj)+'.npy')
  #---------------------------------------------------------------------------------
  # Correr solo el modelo
  P= CNNrITL(d=2, k=len(np.unique(y)), epochs=150, batch_size=64, 
             lr=1e-3, sl='ritl', lk=0.5, l1=1e-3, plot_model=True)
  P.fit(X,y)
  #--------------------------------------------------------------------------------
  # Correr con grid search
  steps = [('proy', CNNrITL(epochs=150, batch_size=64, k=len(np.unique(y)), lr = 1e-3, sl='ritl')),
           ]    
  method = Pipeline(steps ,memory='datospipeline')
  parameters ={ 'proy__l1':[1e-3, 1e-2, 1e-1],
                'proy__lk':[0.5,0.75,1],
                'proy__d':[2,5,10],
                }
  cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=123)
  grid_search = GridSearchCV(method, parameters,cv=cv,verbose=10,
                      scoring='accuracy',refit='accuracy',n_jobs=-1,error_score='raise' )
  grid_search.fit(X, y)
  print(grid_search.best_params_)
  # guardar modelo 
  grid_search.best_estimator_[0].model.save('/content/gauss/Modelos/sbj'+str(sbj)+'.h5')
  print('para el sujeto {} el accuracy  es {}'.format(sbj,grid_search.best_score_))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
30
31
32
33
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
